<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/Exp_7_Log_Reg_with_HPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow boto3 awscli optuna lightgbm imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:

import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': '2CWJKC4QY0Q7GTY8', 'HostId': 'frUHGjqRMlSAtUniJ+Sz9mx5eeIXC1hhQSxDxe4/EOrckoXsUYMcze5mjOV9wxbUomRQr/ZSftc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'frUHGjqRMlSAtUniJ+Sz9mx5eeIXC1hhQSxDxe4/EOrckoXsUYMcze5mjOV9wxbUomRQr/ZSftc=', 'x-amz-request-id': '2CWJKC4QY0Q7GTY8', 'date': 'Fri, 17 Oct 2025 02:56:05 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'adityasahusomu02', 'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [5]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [6]:
mlflow.set_experiment("exp 7 -LogReg with HP Tuning")

2025/10/17 03:01:57 INFO mlflow.tracking.fluent: Experiment with name 'exp 7 -LogReg with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://project1-mlflow-bucket/344689141043668419', creation_time=1760670117580, experiment_id='344689141043668419', last_update_time=1760670117580, lifecycle_stage='active', name='exp 7 -LogReg with HP Tuning', tags={}>

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [8]:
url = 'https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv'
df = pd.read_csv(url).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
df = df.dropna(subset=['category'])

ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")


def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

run_optuna_experiment()


[I 2025-10-17 03:04:37,120] A new study created in memory with name: no-name-4e84eedd-7789-4af3-9d12-198acff7922b
[I 2025-10-17 03:04:37,273] Trial 0 finished with value: 0.7351511308391461 and parameters: {'C': 0.08811896665304415, 'penalty': 'l1'}. Best is trial 0 with value: 0.7351511308391461.
[I 2025-10-17 03:04:37,970] Trial 1 finished with value: 0.774149228492919 and parameters: {'C': 9.11192890607464, 'penalty': 'l2'}. Best is trial 1 with value: 0.774149228492919.
[I 2025-10-17 03:04:38,089] Trial 2 finished with value: 0.6349608962164447 and parameters: {'C': 0.028374587909326773, 'penalty': 'l1'}. Best is trial 1 with value: 0.774149228492919.
[I 2025-10-17 03:04:38,307] Trial 3 finished with value: 0.6963644049883746 and parameters: {'C': 0.025382620555148568, 'penalty': 'l2'}. Best is trial 1 with value: 0.774149228492919.
[I 2025-10-17 03:04:38,564] Trial 4 finished with value: 0.7800676389769605 and parameters: {'C': 0.5852963879674914, 'penalty': 'l1'}. Best is trial 4

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/344689141043668419/runs/fa8ec4517a414b30a0448c1b69825cae
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/344689141043668419


Why use log=True? - Optuna will choose C (in this case) on a logarithmic scale. Optuna has 2 ways of choosing the values: evenly spaced or spread out. If we use log=True, Optuna will pick values that increase by 10× each time, like 0.0001, 0.001, 0.01, 0.1, 1, 10. If you don’t use log=True, Optuna will pick evenly spaced values, like 0.0001, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10
Here, all the small values (below 1) get almost ignored, even though they might be the best ones.

Generally we use log=True in case of small bounded parameters